In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#np.random.seed(4203030)


class material:
    def __init__(self,name:str,sigma_a:float,sigma_s:float,sigma_f:float,nu:float):
        
        self.name=name
        self.sigma_a=sigma_a
        self.sigma_s=sigma_s
        self.sigma_f=sigma_f
        self.nu=nu

        self.sigma_t=self.sigma_a+self.sigma_f+sigma_s


class source:
    def __init__(self,type:str,r:float):
        self.r=r     



def sample_initial_neutrons(number_of_neutrons,source:object):
    """ 
    will accept the number of neutrons and send their sampled x,y position. 
    
    """

    x=np.random.uniform(-source.r,source.r,size=number_of_neutrons)*np.cos(np.random.uniform(0,np.pi*2,size=number_of_neutrons))
    y=np.random.uniform(-source.r,source.r,size=number_of_neutrons)*np.sin(np.random.uniform(0,np.pi*2,size=number_of_neutrons))

    return x,y 
  


In [ ]:


X_left=0
X_right=10

Y_up=10
Y_down=0

Nx=100
Ny=Nx

dx=(X_right-X_left)/Nx
dy=(Y_up-Y_down)/Ny

X = np.linspace(dx*0.5, (X_right-X_left) - 0.5*dx,Nx)
Y = np.linspace(dx*0.5, (Y_up-Y_down) - 0.5*dx,Ny)

X,Y=np.meshgrid(X,Y)

In [ ]:
uo2=material(name='fuel',sigma_a=0.2,sigma_f=.3,sigma_s=3,nu=2.5)
h20=material(name='coolant',sigma_a=.5,sigma_f=0,sigma_s=0.7,nu=0)

source=source(type='neutron',r=2)


def find_mat(pos):

    x=pos[0]
    y=pos[1]
    
    if x**2+y**2<=4:
        return uo2
    elif(x**2+y**2>4) and abs(x)<=7 and abs (y)<=7 :
        return h20
    else:
        return None

In [ ]:
active_batch=100
inactive_batch=50
no_of_gen_per_batch=5
number_of_neutrons=10000

total_batch=active_batch+inactive_batch

scalar_flux = np.zeros((Nx,Ny))
scalar_flux_tl = np.zeros_like (scalar_flux)
weights=np.ones(number_of_neutrons)

In [ ]:

x,y=sample_initial_neutrons(number_of_neutrons=number_of_neutrons,source=source)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(x,y,color='g')
plt.plot(4*np.cos(np.linspace(0,np.pi*2)),4*np.sin(np.linspace(0,np.pi*2)),color='red')
plt.xlim(-10,10)
plt.ylim(-10,10)

In [ ]:
for batch in range (active_batch+inactive_batch):

    for g in range( no_of_gen_per_batch):

        for n in range(number_of_neutrons):

            pos=np.array([x[n],y[n]])
            weight=weights[n]
            alive=True
            
            while alive: 

                ''' sample distance first'''
                material=find_mat(pos)
                rand_num=np.random.rand()
                l=-np.log(rand_num)/material.sigma_t
                
                #sample the angualar direction as well
                mu=np.random.rand()*2-1
                eta=np.sqrt(1-mu**2)

                new_pos=pos+l*np.array(mu,eta)

                if find_mat(new_pos)==None: 
                    alive=False
                else: 
                    '''
                      now we sample the direction 
                    '''
                    reaction_type=np.random.choice('scattering','capture','fission',p=[material.sigma_s/material.sigma_t,material.sigma_a/material.sigma_t,material.sigma_f/material.sigma_t])
                    if reaction_type=='scattering':

                        '''
                            change particle direction by sampling new mu and eta
                        '''
                    elif reaction_type=='capture':
                        ''' 
                            kill the particle
                        '''
                    else: 
                        ''' as there are only three choice the other one will be fission 
                            so adjust particle weight and kill the particle 
                        '''

In [ ]:
np.random.rand()*2-1